In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q ray
!pip install -q flwr[simulation]==1.9.0 tensorflow scikit-learn pandas "numpy<2"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import flwr as fl
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
X = pd.read_csv("/content/drive/MyDrive/UNSW_X_preprocessed.csv")
y = pd.read_csv("/content/drive/MyDrive/UNSW_y.csv")

# If y is a DF with one column
if y.shape[1] == 1:
    y = y.iloc[:, 0]

X = X.values.astype("float32")
y = y.values.astype("int32")

print(X.shape, y.shape)


(82332, 44) (82332,)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [4]:
X_train_all, X_test, y_train_all, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, stratify=y
)

print("Train:", X_train_all.shape, "Test:", X_test.shape)


Train: (69982, 44) Test: (12350, 44)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
NUM_CLIENTS = 40

# Shuffle indices
rng = np.random.RandomState(42)
indices = np.arange(len(X_train_all))
rng.shuffle(indices)

client_splits = np.array_split(indices, NUM_CLIENTS)

client_data = []
for i, idxs in enumerate(client_splits):
    client_data.append((X_train_all[idxs], y_train_all[idxs]))
    print(f"Client {i} -> {len(idxs)} samples")


Client 0 -> 1750 samples
Client 1 -> 1750 samples
Client 2 -> 1750 samples
Client 3 -> 1750 samples
Client 4 -> 1750 samples
Client 5 -> 1750 samples
Client 6 -> 1750 samples
Client 7 -> 1750 samples
Client 8 -> 1750 samples
Client 9 -> 1750 samples
Client 10 -> 1750 samples
Client 11 -> 1750 samples
Client 12 -> 1750 samples
Client 13 -> 1750 samples
Client 14 -> 1750 samples
Client 15 -> 1750 samples
Client 16 -> 1750 samples
Client 17 -> 1750 samples
Client 18 -> 1750 samples
Client 19 -> 1750 samples
Client 20 -> 1750 samples
Client 21 -> 1750 samples
Client 22 -> 1749 samples
Client 23 -> 1749 samples
Client 24 -> 1749 samples
Client 25 -> 1749 samples
Client 26 -> 1749 samples
Client 27 -> 1749 samples
Client 28 -> 1749 samples
Client 29 -> 1749 samples
Client 30 -> 1749 samples
Client 31 -> 1749 samples
Client 32 -> 1749 samples
Client 33 -> 1749 samples
Client 34 -> 1749 samples
Client 35 -> 1749 samples
Client 36 -> 1749 samples
Client 37 -> 1749 samples
Client 38 -> 1749 samp

In [6]:
def create_model(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')  # binary classification
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

input_dim = X.shape[1]


In [7]:
class FLClient(fl.client.NumPyClient):
    def __init__(self, X, y):
        self.model = create_model(input_dim)
        self.X = X
        self.y = y

    def get_parameters(self, config=None):
        return self.model.get_weights()

    def set_parameters(self, parameters):
        self.model.set_weights(parameters)

    def fit(self, parameters, config=None):
        self.set_parameters(parameters)
        self.model.fit(self.X, self.y, epochs=2, batch_size=64, verbose=0)
        return self.get_parameters(), len(self.X), {}

    def evaluate(self, parameters, config=None):
        self.set_parameters(parameters)
        loss, acc = self.model.evaluate(self.X, self.y, verbose=0)
        return loss, len(self.X), {"accuracy": acc}


In [8]:
def client_fn(cid):
    cid = int(cid)
    Xc, yc = client_data[cid]
    return FLClient(Xc, yc)

strategy = fl.server.strategy.FedAvg(
    min_fit_clients=NUM_CLIENTS,
    min_available_clients=NUM_CLIENTS,
    fraction_fit=1.0
)

history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=5),
    strategy=strategy,
)


INFO :      Starting Flower simulation, config: num_rounds=5, no round_timeout
2025-11-16 07:49:04,120	INFO worker.py:2012 -- Started a local Ray instance.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py:2051: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'object_store_memory': 3928097587.0, 'memory': 9165561037.0}


In [9]:
# Collect final client weights
weights_list = []
for c in range(NUM_CLIENTS):
    Xc, yc = client_data[c]
    client = FLClient(Xc, yc)
    weights_list.append(client.get_parameters())

# Average weights
avg_weights = []
for weights in zip(*weights_list):
    avg_weights.append(np.mean(np.array(weights), axis=0))

global_model = create_model(input_dim)
global_model.set_weights(avg_weights)

print("Global model built from aggregated client weights.")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Global model built from aggregated client weights.


In [10]:
pred_probs = global_model.predict(X_test)
preds = np.argmax(pred_probs, axis=1)

print(classification_report(y_test, preds, digits=4))

acc = accuracy_score(y_test, preds)
prec = precision_score(y_test, preds, zero_division=0)
rec = recall_score(y_test, preds, zero_division=0)
f1 = f1_score(y_test, preds, zero_division=0)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1:", f1)


386/386 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
              precision    recall  f1-score   support

           0     0.4733    0.5546    0.5107      5550
           1     0.5772    0.4963    0.5337      6800

    accuracy                         0.5225     12350
   macro avg     0.5253    0.5255    0.5222     12350
weighted avg     0.5305    0.5225    0.5234     12350

Accuracy: 0.5225101214574899
Precision: 0.5772190867111339
Recall: 0.4963235294117647
F1: 0.5337234126670357


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [11]:
results_df = pd.DataFrame(X_test, columns=[f"f{i}" for i in range(X_test.shape[1])])
results_df["true"] = y_test
results_df["pred"] = preds
results_df["action"] = results_df["pred"].apply(lambda x: "BLOCK" if x == 1 else "ALLOW")

blocked = results_df[results_df["action"] == "BLOCK"]
blocked_path = "/content/drive/MyDrive/UNSW_blocked_predictions_FL2.csv"
blocked.to_csv(blocked_path, index=False)

print(f"Blocked flows saved to: {blocked_path}")
print("Total blocked:", len(blocked))
print("True attacks blocked:", sum((results_df["true"]==1) & (results_df["action"]=="BLOCK")))
print("False blocks:", sum((results_df["true"]==0) & (results_df["action"]=="BLOCK")))


Blocked flows saved to: /content/drive/MyDrive/UNSW_blocked_predictions_FL2.csv
Total blocked: 5847
True attacks blocked: 3375
False blocks: 2472


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [12]:
blocked.head(10)


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f37,f38,f39,f40,f41,f42,f43,true,pred,action
0,0.596010,-0.213730,0.410563,0.274329,0.932695,-0.124455,-0.151816,-0.045909,-0.087369,6.174084,...,-0.090857,-0.090617,-0.203143,-0.288904,-0.194604,-0.10607,0.008552,1,1,BLOCK
4,-0.274349,-0.213533,0.410563,0.274329,-2.050741,-0.124455,-0.134511,-0.045816,-0.086299,-0.532781,...,-0.090857,-0.090617,-0.203143,-0.171861,-0.464356,-0.10607,0.707105,0,1,BLOCK
7,0.217968,0.002373,0.088625,1.697431,-0.559023,0.114501,1.319111,-0.036110,1.345788,-0.553168,...,-0.090857,-0.090617,1.362590,-0.640033,-0.734107,-0.10607,-2.087104,1,1,BLOCK
8,0.946409,3.749871,-1.735686,-0.674406,0.932695,0.009958,-0.151816,-0.041330,-0.087369,-0.554503,...,-0.090857,-0.090617,-0.203143,-0.405947,-0.554273,-0.10607,-1.388552,1,1,BLOCK
9,1.715326,-0.069040,0.088625,-0.674406,-0.559023,-0.064716,-0.082596,-0.043462,-0.085032,-0.554342,...,-0.090857,-0.090617,-0.203143,-0.640033,-0.644190,-0.10607,0.707105,0,1,BLOCK
11,-1.056856,-0.213730,0.410563,0.274329,0.932695,-0.124455,-0.151816,-0.045909,-0.087369,6.174084,...,-0.090857,-0.090617,-0.203143,2.637169,1.783574,-0.10607,0.008552,1,1,BLOCK
12,0.613640,-0.213727,0.410563,-0.674406,0.932695,-0.124455,-0.151816,-0.039454,-0.087369,0.006207,...,-0.090857,-0.090617,-0.203143,0.062225,0.614650,-0.10607,-0.690000,1,1,BLOCK
16,0.015294,-0.012240,0.088625,-0.674406,-2.050741,-0.094586,-0.134511,-0.040759,-0.086801,-0.554460,...,-0.090857,-0.090617,-0.203143,-0.640033,-0.644190,-0.10607,0.707105,0,1,BLOCK
20,0.639179,-0.213728,-3.452685,-0.674406,0.932695,-0.124455,-0.151816,-0.045408,-0.087369,0.406718,...,-0.090857,-0.090617,-0.203143,-0.171861,0.344899,-0.10607,-0.690000,1,1,BLOCK
21,0.816146,-0.213729,0.410563,0.274329,0.932695,-0.124455,-0.151816,-0.045909,-0.087369,1.127639,...,-0.090857,-0.090617,-0.203143,0.179268,0.434816,-0.10607,0.008552,1,1,BLOCK


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
